# Passive simulation of a robot in pydrake

*Copyright (c) 2018, Toyota Research Institute
All rights reserved.*

See license in `LICENSE`

## Introduction
This notebook shows how to create and run a simulation of a robot using Drake's systems framework.
Please see the README for prerequisites.

In [ ]:
import os.path
from pydrake.multibody.parsers import PackageMap
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.rigid_body_tree import (
    AddModelInstanceFromUrdfStringSearchingInRosPackages,
    FloatingBaseType,
    RigidBodyFrame,
    RigidBodyTree,
)
from pydrake.lcm import DrakeLcm
from pydrake.systems.framework import BasicVector, DiagramBuilder
from pydrake.systems.analysis import Simulator
from pydrake.systems.primitives import ConstantVectorSource
from pydrake.multibody.rigid_body_plant import (
    DrakeVisualizer,
    RigidBodyPlant
)
import numpy as np
import ipywidgets as widgets

## Demo

Build a diagram containing a `RigidBodyPlant` and a `DrakeVisualizer`.

In [ ]:
iiwa_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/urdf/iiwa14_spheres_collision.urdf")

builder = DiagramBuilder()

# Add the sub-systems
plant = builder.AddSystem(
    RigidBodyPlant(tree=RigidBodyTree(iiwa_file, FloatingBaseType.kFixed),
                   timestep=1e-3))
zero_torque_source = builder.AddSystem(ConstantVectorSource(np.zeros(plant.get_num_actuators())))
visualizer = builder.AddSystem(
    DrakeVisualizer(tree=plant.get_rigid_body_tree(), 
                    lcm=DrakeLcm(), 
                    enable_playback=True))

# Connect the sub-systems
# Torque source -> plant
builder.Connect(zero_torque_source.get_output_port(0), plant.get_input_port(0))
# Plant -> visualizer
builder.Connect(plant.get_output_port(0), visualizer.get_input_port(0))

# Build the diagram.
diagram = builder.Build()

Create a simulator for the diagram.

In [ ]:
simulator = Simulator(diagram)
simulator.Initialize()

Simulate for 10 seconds (as quickly as possible).

In [ ]:
simulator.StepTo(10)

Replay the simulation results at 1x real time.

In [ ]:
visualizer.ReplayCachedSimulation()